In [80]:
from selenium import webdriver
import time
import os
from gsheets import Sheets
import pandas as pd
import operator
import shutil
from shutil import copyfile
import os.path
import csv
from gspread_pandas import Spread, Client 
from oauth2client.service_account import ServiceAccountCredentials
import pprint
import gspread
import math
from selenium.webdriver.chrome.options import Options
import getpass
import re
import goslate
from random import randint
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from random import randint
import random

In [81]:
##removing the annoying tab that toggles and says something like: chrome is monitoring automated something something 
options = Options()
options.add_argument("start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

In [82]:
os.chdir('/Users/'+getpass.getuser()+'/Desktop/auto-rego')
f=pd.read_csv('input.csv').event[0]
f

'Fujitsu rego page'

In [83]:
#reading credientials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

#wd for authentiction file
os.chdir('/Users/'+getpass.getuser()+'/Desktop/auto-rego')
creds = ServiceAccountCredentials.from_json_keyfile_name('linkupdater-631ef6e77556.json', scope)

In [84]:
#getting permission 
client = gspread.authorize(creds)

In [85]:
statedictionary=['Select...', 'ACT', 'NSW', 'NT', 'NZ', 'QLD', 'SA', 'VIC', 'WA', 'TAS']

industrydictionary=['Select...',
 'Commercial Sector',
 'Defence',
 'Health',
 'Public Safety',
 'Public Sector',
 'Retail']

iamdictionary=['Select...',
 'Customer',
 'Customer (Public Sector)',
 'Channel-Partner/Distributor',
 'Technology Partner',
 'Other Partners',
 'Event Service',
 'Press/Blogger',
 'Analyst',
 'Fujitsu Employee',
 'Prospect/Interested Party',
 'Supplier',
 'University Student',
 'Other']

#the below 
#for i in range(0,10):
#    print(web.find_element_by_xpath('//*[@id="zOPprogressiveprofilingsprov"]/option['+str(i+1)+']').text)

In [86]:
web = webdriver.Chrome(options=options, executable_path='/Users/'+getpass.getuser()+'/Desktop/auto-rego/chromedriver')

web.get("https://marketing-oceania.global.fujitsu.com/activatenow21-reg?utm_source=Third_Party&utm_medium=Social&utm_campaign=ActivateNow_Oceania&utm_content=Innovatus")       

In [87]:
#opening production sheet
sheet= client.open(f).worksheet("Registration sheet")

df = pd.DataFrame(sheet.get_all_records())
harvestfield=df[df.Registered == 'need1'].reset_index(drop=True)
len(harvestfield)
harvestfield

,First,Last,Email,Company,Title,State,Industry,Attendeetype,Mobile,Registered
0,Amy,Lai,alai@hesta.com.au,Hesta,Experience Design Lead,VIC,Commercial Sector,Other,61 3 8662 2573,need1


In [88]:
##############################
#email column on google sheets
##############################
emailloc=2
statusloc=10

##the below code fills in the registration page and submits it
for i in range(len(harvestfield.Email)):        
    print(str(i)+" start check")
    
    try:
        web.get("https://marketing-oceania.global.fujitsu.com/activatenow21-reg?utm_source=Third_Party&utm_medium=Social&utm_campaign=ActivateNow_Oceania&utm_content=Innovatus")       
        
        #first
        first=harvestfield.First[i]
        firstslot=web.find_element_by_xpath('//*[@id="FirstName"]')
        time.sleep(random.uniform(0,1))
        firstslot.send_keys(first)
        time.sleep(random.uniform(0,1))

        #last
        last=harvestfield.Last[i]
        lastslot=web.find_element_by_xpath('//*[@id="LastName"]')
        time.sleep(random.uniform(0,1))
        lastslot.send_keys(last)
        time.sleep(random.uniform(0,1))

        #email
        email=harvestfield.Email[i]
        emailslot=web.find_element_by_xpath('//*[@id="Email"]')
        time.sleep(random.uniform(0,1))
        emailslot.send_keys(email)
        time.sleep(random.uniform(0,1))

        #company
        organisation=harvestfield.Company[i]
        organisationslot=web.find_element_by_xpath('//*[@id="Company"]')
        time.sleep(random.uniform(0,1))
        organisationslot.send_keys(organisation)
        time.sleep(random.uniform(0,1))

        #title
        title=harvestfield.Title[i]
        titleslot=web.find_element_by_xpath('//*[@id="Title"]')
        time.sleep(random.uniform(0,1))
        titleslot.send_keys(title)
        time.sleep(random.uniform(0,1))

        #state
        state=int(statedictionary.index(harvestfield.State[i]))        
        statebutton=web.find_element_by_xpath('//*[@id="State_Province__c"]/option['+str(state+1)+']')                                                                                                            
        time.sleep(random.uniform(0,1))
        statebutton.click()              
        time.sleep(random.uniform(0,1))

        #industry
        industry=int(industrydictionary.index(harvestfield.Industry[i]))        
        industrybutton=web.find_element_by_xpath('//*[@id="Industry_Group__c"]/option['+str(industry+1)+']')                                                                                                            
        time.sleep(random.uniform(0,1))
        industrybutton.click()              
        time.sleep(random.uniform(0,1))

        #iam
        iam=int(iamdictionary.index(harvestfield.Attendeetype[i]))        
        iambutton=web.find_element_by_xpath('//*[@id="LeadRole"]/option['+str(iam+1)+']')                                                                                                            
        time.sleep(random.uniform(0,1))
        iambutton.click()              
        time.sleep(random.uniform(0,1))

        #number
        contactnumber=re.sub("[^0-9]", "",str(harvestfield.Mobile[i]))
        contactnumberslot=web.find_element_by_xpath('//*[@id="MobilePhone"]')
        time.sleep(random.uniform(0,1))
        contactnumberslot.send_keys(str(contactnumber))
        time.sleep(random.uniform(0,1))

        #sub button
        time.sleep(random.uniform(0,1))
        subbutton=web.find_element_by_xpath('//*[@id="programField1"]')
        web.execute_script("arguments[0].click();", subbutton)

        #sub button2
        time.sleep(random.uniform(0,1))
        subbutton2=web.find_element_by_xpath('//*[@id="programField2"]')
        web.execute_script("arguments[0].click();", subbutton2)

        #sub button3
        time.sleep(random.uniform(0,1))
        subbutton3=web.find_element_by_xpath('//*[@id="mktoForm_705"]/div[14]/span/button')
        web.execute_script("arguments[0].click();", subbutton3)
                
        time.sleep(random.uniform(4,7))                
    except:
        #problem case (when country input has a problem)
        sheet.update_cell(int(df[df.iloc[:,emailloc]==harvestfield.Email[i]].index.to_numpy())+2,statusloc,"problem")
        
        time.sleep(random.uniform(5,7))
        continue

    #success check    
    try:
        web.find_element_by_xpath('//*[@id="bodyId"]/div[4]/div[1]/div/div/div/h2') 
        #updating pm sheet        
        sheet.update_cell(int(df[df.iloc[:,emailloc]==harvestfield.Email[i]].index.to_numpy())+2,statusloc,"yes - regoed")        
        time.sleep(random.uniform(5,7))
        continue 
               
    except:    
        print("")
        #success check    
    try:    
        web.find_element_by_xpath('//*[@id="sfid-3116053242"]/a')                          
        #updating pm sheet        
        sheet.update_cell(int(df[df.iloc[:,emailloc]==harvestfield.Email[i]].index.to_numpy())+2,statusloc,"yes - regoed")        
        time.sleep(random.uniform(5,7))
        continue 
                        
    except:
        sheet.update_cell(int(df[df.iloc[:,emailloc]==harvestfield.Email[i]].index.to_numpy())+2,statusloc,"problem")
        time.sleep(random.uniform(5,7))
        continue   

0 start check


In [89]:
web.close()